In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

print('Workdir: ', work_dir)

Workdir:  /home/carol/etl/workshop-02


In [2]:
from db.db_connection import build_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect
from models.model import GrammyAward
from sqlalchemy.exc import SQLAlchemyError
from src.transform.transform_grammy import TransformGrammy

In [3]:
engine = build_engine()

Successfully connected to the database workshop2!


In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
try:
    inspector = inspect(engine)

    if inspector.has_table('grammy_awards'):
        try:
            GrammyAward.__table__.drop(engine)
        except SQLAlchemyError as e:
            print(f"Error dropping table: {e}")
            raise
    try:
        GrammyAward.__table__.create(engine)
        print("Table creation was successful.")
    except SQLAlchemyError as e:
        print(f"Error creating table: {e}")
        raise

except SQLAlchemyError as error:
    print(f"An error occurred: {error}")

Table creation was successful.


In [6]:
try:
    df = TransformGrammy('../data/the_grammy_awards.csv')
    
    df.insert_ids()
    
    df.df.to_sql('grammy_awards', engine, if_exists='append', index=False)
    
    print("Data uploaded successfully")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if session:
        session.close()

Data uploaded successfully
